In [1]:
# from annotation_utils import *
import annotation_utils as utils
import numpy as np
import cv2

In [2]:
keypoints, anchors = utils.get_points_from_CAD()

normalized_keypoints = utils.normalize_and_orient_kpts(keypoints)
normalized_anchors = utils.normalize_and_orient_kpts(anchors)

In [7]:
#  calibration procedure is performed on warped image (tentative)
cv2.destroyAllWindows()
camera_params = [633.0902963939839, 638.754439101168,
                 629.0646296262861, 362.7401326185789]

anchor_ground_truth = np.array(
[(597, 349),
 (579, 325),
 (614, 324),
 (562, 350),
 (633, 349),
 (581, 380),
 (615, 379)])

img_with_marker_name = 'marker.jpg'
img_without_marker_name = 'no_marker.jpg'

img = cv2.imread(img_with_marker_name)

res = utils.detect_apriltag(img, camera_params)
img_warped, desired_corners,tform = utils.warp_img(img, res)

res_warp = utils.detect_apriltag(img_warped,camera_params)

anchor_on_image = utils.get_keypoints_on_warped_image(res_warp, normalized_anchors)
print("anchor difference in pix")
print(anchor_on_image-anchor_ground_truth)

# calculates corrected points from 1 shot
corrected_points, corrected_normalized_keypoints,transformed_anchors_points = utils.get_corrected_normalized_keypoints(res_warp,
                                        normalized_anchors,
                                        anchor_ground_truth,
                                        normalized_keypoints)

cv2.destroyAllWindows()

anchor difference in pix
[[-0.980896    3.49934387]
 [-0.97610474  2.30605164]
 [-0.98568726  2.30605164]
 [-0.97131348  2.49934387]
 [-0.99047852  2.49934387]
 [-0.97610474  3.01354301]
 [-0.98568726  3.01354301]]
tform.params
[[ 1.         0.        -0.980896 ]
 [ 0.         1.         2.7338887]
 [ 0.         0.         1.       ]]
resulting transform to correct points
[[1.00036948e+00 7.20941705e-16 7.51073990e-01]
 [8.66677684e-16 9.83088179e-01 3.01163202e+00]]
inliers [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]
[[613.01761719 275.88550954]
 [632.98238281 275.88550954]
 [578.77884038 290.00875414]
 [667.22115962 290.00875414]
 [601.61335826 292.89280884]
 [644.38664174 292.89280884]
 [577.97608976 305.02947313]
 [668.02391024 305.02947313]
 [560.83882634 326.95531532]
 [685.16117366 326.95531532]
 [590.66247469 379.33939409]
 [655.33752531 379.33939409]
 [589.84997405 382.72009387]
 [656.15002595 382.72009387]]


Error, more than one new minimum found.


In [5]:
img_without_marker_name = "no_marker_close_new_4.jpg"
img = cv2.imread(img_without_marker_name)

cv2.imshow("q",img)
cv2.waitKey(0)
# np.zeros((10,2))
# show_image_with_points(img,anchor_ground_truth, "no_correction")

32

In [6]:
cv2.imshow('draw_points_on_aruco', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
normalized_keypoints - corrected_normalized_kpts 

In [9]:
def show_image_with_points(img,points, comment):
    for c in points:
        kc = int(c[0]), int(c[1])
        cv2.circle(img, kc, 1, (0,255, 255), -1)
    cv2.imshow(comment, img)
    cv2.waitKey(0)
    cv2.imwrite(comment+".jpg", img)

img_with_marker_name = 'marker.jpg'
img_without_marker_name = 'no_marker.jpg'


    # visualize the difference between corrected and original points
img = cv2.imread(img_with_marker_name)
points_on_image_corrected = utils.get_keypoints_on_image(img, corrected_normalized_keypoints,camera_params)
points_on_image = utils.get_keypoints_on_image(img,normalized_keypoints, camera_params)
                                                                                                                                                                                                                                                                                                            
img = cv2.imread(img_without_marker_name)
show_image_with_points(img,points_on_image, "no_correction")

img = cv2.imread(img_without_marker_name)
show_image_with_points(img,points_on_image_corrected, "correction points")

cv2.destroyAllWindows()

[[ 1.00316457e+00 -1.33530746e-02  4.84560574e+00]
 [ 1.31839624e-02  9.87511111e-01 -2.41836216e+00]
 [ 8.78849449e-07 -2.73141213e-07  1.00000000e+00]]
[[ 1.00316457e+00 -1.33530746e-02  4.84560574e+00]
 [ 1.31839624e-02  9.87511111e-01 -2.41836216e+00]
 [ 8.78849449e-07 -2.73141213e-07  1.00000000e+00]]


In [11]:
cv2.destroyAllWindows()

In [ ]:

img = cv2.imread(img_with_marker_name)
show_image_with_points(img,points_on_image, "kpts_on_marker")

In [ ]:
import cv2
import numpy as np

source = np.float32([[0, 0], [100, 0], [100, 100], [0, 100]])
dest = np.float32([[0, 34], [-1000,15], [-1000, -1000], [16, -1000]])

points = np.float32([[[50, 50]]])

homography, _ = cv2.findHomography(source, dest)
print(homography)
transformed = cv2.perspectiveTransform(points, homography)

print(transformed)
# => [[[-500. -500.]]]

homography_inverse = np.linalg.inv(homography)
print(homography_inverse)
detransformed = cv2.perspectiveTransform(transformed, homography_inverse)
print(homography@homography_inverse)
print(detransformed)
# => [[[50. 50.]]]

In [ ]:
# reserved data
# from canny edge and hough circles
anchor_ground_truth = np.array(
    [[494.25,444.75,44.15],
  [636.75,362.25,42.5],
  [590.25,444.75,42.5],
  [537.75,360.75,47.45],
  [492.75,300.75,30.95],
  [450.75, 366.75,39.2],
  [582.75,299.25,30.95]])


anchor_ground_truth = np.array(
[(597, 349),
 (579, 325),
 (614, 324),
 (562, 350),
 (633, 349),
 (581, 380),
 (615, 379)])

img = cv2.imread('marker.jpg')